# APIM ❤️ OpenAI

## GPT-4o inferencing lab
![flow](../../images/GPT-4o-inferencing.gif)

Playground to try the new GPT-4o model. GPT-4o ("o" for "omni") is designed to handle a combination of text, audio, and video inputs, and can generate outputs in text, audio, and image formats.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk1)
- [🧪 Image Processing](#sdk2)
- [🧪 Base64 Image Processing](#sdk3)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [1]:
import os
import json
import datetime
import requests

lab_prefix="aig-pevo1-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style

resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
#apim_resource_sku = "Basicv2"
apim_resource_sku = "Consumption" 

openai_resources = [ {"name": "openai1", "location": "westus3"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-4o"
openai_model_version = "2024-05-13"
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": f"https://{tool_prefix}openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": f"https://{tool_prefix}openaimock2.azurewebsites.net"} ]

log_analytics_name = "workspace"
app_insights_name = 'insights'

print("✅ Set Variables done ⌚ ", datetime.datetime.now().isoformat())


✅ Set Variables done ⌚  2024-10-26T12:48:19.382853


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [2]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  aig-pevo1-GPT-4o-inferencing  created ⌚  12:48:27.719004


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)

print("✅ bicep based deployment done ⌚ ", datetime.datetime.now().isoformat())



/home/per/azdevops/pevosdcgm/AI-Gateway/labs/GPT-4o-inferencing/main.bicep(282,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]
/home/per/azdevops/pevosdcgm/AI-Gateway/labs/GPT-4o-inferencing/main.bicep(299,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]

{- Finished .. - Starting ..
  "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aig-pevo1-GPT-4o-inferencing/providers/Microsoft.Resources/deployments/GPT-4o-inferencing",
  "location": null,
  "name": "GPT-4o-inferencing",
  "properties": {
    "correlationId": "9a1dfc8c-d1c4-44

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [4]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)



👉🏻 API Gateway URL:  https://apim-wksiztrbb7ubs.azure-api.net
👉🏻 Workspace ID:  052ba12a-f90d-458e-97b9-cb236a9b7c57
👉🏻 App ID:  9abe2894-2fe2-43c4-9f06-7944468fa185


<a id='requsts'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [5]:
import time
runs = 1
sleep_time_ms = 1000

url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)


▶️ Run:  1
status code:  200
headers  {'Content-Type': 'application/json', 'Date': 'Sat, 26 Oct 2024 10:54:36 GMT', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-ms-region': 'West US 3', 'apim-request-id': '66012b36-1e48-46c3-9059-83b5431649ba', 'x-ratelimit-remaining-requests': '19', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': '7f26ca8c-4e89-442c-9fcb-072a4ad9061f', 'azureml-model-session': 'd038-20240925071838', 'X-Content-Type-Options': 'nosniff', 'x-envoy-upstream-service-time': '350', 'x-ms-client-request-id': 'Not-Set', 'x-ratelimit-remaining-tokens': '19340', 'Request-Context': 'appId=cid-v1:9abe2894-2fe2-43c4-9f06-7944468fa185'}
x-ms-region:  West US 3
response:  Sure, let me just check my invisible watch for you. Oh, wait, I forgot—it doesn't exist. Maybe try looking at a clock?


<a id='sdk1'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK


In [6]:
import time
from openai import AzureOpenAI
runs = 1
sleep_time_ms = 1000

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    print("💬 ", response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


▶️ Run:  1
💬  Oh, sure! Let me just whip out my magical clock that tells the time for everyone, everywhere. Oh wait, I don't have one. How about you just check your own clock?


<a id='sdk2'></a>
### 🧪 Image Processing

GPT-4o can directly process images and take intelligent actions based on the image. 

The following example is adapted from the [OpenAI Cookbook](https://cookbook.openai.com/examples/gpt4o/introduction_to_gpt4o)

#### Input image:
![image](https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png)





In [7]:
from IPython.display import display, Markdown
response = client.chat.completions.create(
    model=openai_model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that responds in Markdown. Help me with my math homework!"},
        {"role": "user", "content": [
            {"type": "text", "text": "What's the area of the triangle?"},
            {"type": "image_url", "image_url": {
                "url": "https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png"}
            }
        ]}
    ],
    temperature=0.0,
)

display(Markdown(response.choices[0].message.content))


To find the area of the triangle, we can use Heron's formula. Heron's formula states that the area of a triangle with sides of length \(a\), \(b\), and \(c\) is:

\[ \text{Area} = \sqrt{s(s-a)(s-b)(s-c)} \]

where \(s\) is the semi-perimeter of the triangle:

\[ s = \frac{a + b + c}{2} \]

In this triangle, the sides are \(a = 6\), \(b = 5\), and \(c = 9\).

First, calculate the semi-perimeter \(s\):

\[ s = \frac{6 + 5 + 9}{2} = \frac{20}{2} = 10 \]

Now, apply Heron's formula:

\[ \text{Area} = \sqrt{10(10-6)(10-5)(10-9)} \]
\[ \text{Area} = \sqrt{10 \cdot 4 \cdot 5 \cdot 1} \]
\[ \text{Area} = \sqrt{200} \]
\[ \text{Area} = 10\sqrt{2} \]

So, the area of the triangle is \(10\sqrt{2}\) square units.

<a id='sdk3'></a>
### 🧪 Base64 Image Processing

#### Input image:
<img src="meme.png" width="600px">


In [8]:
from IPython.display import display, Markdown, HTML
import base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
base64_image = encode_image("meme.png")
response = client.chat.completions.create(
    model=openai_model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that helps to interprete memes!"},
        {"role": "user", "content": [
            {"type": "text", "text": "What's funny about this picture?"},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
            }
        ]}
    ],
    temperature=0.0,
)
HTML('<font size="5">' + response.choices[0].message.content + '</font>')

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.